In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [7]:
url = "https://sportsbook.draftkings.com/leagues/basketball/nba?category=player-combos&subcategory=pts-+-reb-+-ast"
req = requests.get(url)
soup = bs(req.content, 'lxml')

In [8]:
games_lines = soup.find_all('div', {'class': 'sportsbook-event-accordion__wrapper expanded'})

In [9]:
players = []
for idx, game_line in enumerate(games_lines):
    game = game_line.find_all('tr')
    
    if game:
        print(f'Game {idx}')
        print(f'# Players: {len(game)}')
        
        for player_line in game[1:]:
            player = {}
            player['Name'] = player_line.find('div', 
                                              {'class': 'sportsbook-row-name__wrapper'}
                                             ).find('span',
                                                   {'class': 'sportsbook-row-name'}).text
            player_lines = player_line.find_all('td',
                                             {'class': 'sportsbook-table__column-row'}
                                            )
            if player_lines:
                for line in player_lines:
                    bet_line = line.find('span',
                                        {'class': 'sportsbook-outcome-cell__line'}
                                        ).text
                    player['Bet'] = float(bet_line)
                    
                    label = line.find('span',
                                      {'class': 'sportsbook-outcome-cell__label'}
                                      ).text
                    player[label] = line.find('span',
                                             {'class': 'sportsbook-odds american default-color'}
                                             ).text
            
            players.append(player)

Game 0
# Players: 8
Game 1
# Players: 10
Game 2
# Players: 9
Game 3
# Players: 4


In [10]:
pd.DataFrame(players).to_csv('NBA_Odds.csv', index=0)